In [174]:
import pandas as pd
import json
import requests
import logging
from pandas.core.common import flatten

logging.basicConfig(level=logging.WARNING)

class ApiError(Exception):
    """
    Exception raised for API errors.
    Attributes:
        message - explanation of the error
    """
    def __init__(self, message):
        self.message = message
    
api_url_base = 'https://data.police.uk/api'
headers = {'Content-Type': 'application/json'}

def get_police_forces(api_url_base, headers):
    """
    Gets all police forces
    """
    api_url_request = '/forces'
    
    response = requests.get(f'{api_url_base}{api_url_request}', headers=headers)
    if response.status_code != 200:
        raise ApiError(f'GET {api_url_request} {response.status_code}')
    
#     loads response json into a pandas dataframe
    result_response = json.loads(response.content.decode('utf-8'))
    df_result = pd.DataFrame(result_response)
    
    return df_result

def get_social_engagement_methods(api_url_base, headers, police_force):
    """
    Gets social engagement methods for the specified police_force 
    """
    api_url_request = f'/forces/{police_force}'
    
    response = requests.get(f'{api_url_base}{api_url_request}', headers=headers)
    if response.status_code != 200:
        raise ApiError(f'GET {api_url_request} {response.status_code}')
    
#     loads response json into a pandas dataframe
    result_response = json.loads(response.content.decode('utf-8'))['engagement_methods']
    df_result = pd.DataFrame(result_response)
    
    return df_result
    
def has_social_engagement_method(api_url_base, headers, social_engagement_method, police_force):
    """
    Checks if a police force has specified social engagement method
    """
    df_social_engagement_methods = get_social_engagement_methods(api_url_base, headers, police_force)
#     gets all values for the social engagement methods
    list_of_dataframe_string_values = flatten(df_social_engagement_methods.values)
#     filters out None values
    filtered_list = filter(None, list_of_dataframe_string_values)
#     checks if there is any specified social among retrieved values
#     this is needed, since not all police forces have a specified engagement method of type
    has_result = any([social_engagement_method.lower() in str.lower() for str in filtered_list])
    return has_result
    
def get_police_forces_by_social_engagement_method(api_url_base, headers, social_engagement_method):
    """
    Gets all police forces that have the specified social engagement method
    """
#     gets all police forces
    df_police_forces = get_police_forces(api_url_base, headers)
#     adds a column with bool values if a police force has the specified social engagement method
    df_police_forces[social_engagement_method] = df_police_forces.apply(lambda x: has_social_engagement_method(api_url_base, headers, social_engagement_method, x['id']), axis = 1)
#     filters out rows with False value
    df_police_forces = df_police_forces[df_police_forces[social_engagement_method]]
#     drops the social_engagement_method from the return values
    df_police_forces.drop(columns = [social_engagement_method], inplace = True)
#     resets the index
    df_police_forces.reset_index(drop = True, inplace = True)
        
    return df_police_forces

In [137]:
"""
Retrieves all police forces
"""
logger = logging.getLogger(f'all police forces')
logger.setLevel(logging.INFO)

try:    
    df_police_forces = get_police_forces(api_url_base, headers)
    logger.info(f'\n{df_police_forces}')
    
except ApiError as e:
    logger.exception(f'API Error: {e}')

INFO:all police forces:
                    id                                name
0    avon-and-somerset      Avon and Somerset Constabulary
1         bedfordshire                 Bedfordshire Police
2       cambridgeshire         Cambridgeshire Constabulary
3             cheshire               Cheshire Constabulary
4       city-of-london               City of London Police
5            cleveland                    Cleveland Police
6              cumbria                Cumbria Constabulary
7           derbyshire             Derbyshire Constabulary
8   devon-and-cornwall             Devon & Cornwall Police
9               dorset                       Dorset Police
10              durham                 Durham Constabulary
11         dyfed-powys                  Dyfed-Powys Police
12               essex                        Essex Police
13     gloucestershire        Gloucestershire Constabulary
14  greater-manchester           Greater Manchester Police
15               gwent          

In [139]:
"""
Retrieves social engagement methods of a specified police force
"""
logger = logging.getLogger(f'social engagement methods')
logger.setLevel(logging.INFO)

police_force = 'cheshire'

try:    
    df_police_force_social_engagement_methods = get_social_engagement_methods(api_url_base, headers, police_force)
    logger.info(f'\n{df_police_force_social_engagement_methods}')
    
except ApiError as e:
    logger.exception(f'API Error: {e}')

INFO:social engagement methods:
                                       url      type description     title
0  https://www.facebook.com/cheshirepolice  facebook        None  facebook
1   https://www.twitter.com/cheshirepolice   twitter        None   twitter
2           https://www.cheshire.police.uk       web        None       web


In [142]:
"""
Checks if a police force has specified social engagement method 
"""
logger = logging.getLogger(f'has a social engagement method')
logger.setLevel(logging.INFO)

police_force = 'bedfordshire'
social_engagement_method = 'facebook'

try:    
    has_result = has_social_engagement_method(api_url_base, headers, social_engagement_method, police_force)
    logger.info(f'\n{has_result}')
    
except ApiError as e:
    logger.exception(f'API Error: {e}')

INFO:has a social engagement method:
True


In [173]:
"""
Retrieves police forces that have specified social engagement method 
"""
logger = logging.getLogger(f'police forces with specified social engagement method')
logger.setLevel(logging.INFO)

social_engagement_method = 'facebook'

try:    
    police_forces_result = get_police_forces_by_social_engagement_method(api_url_base, headers, social_engagement_method)
    logger.info(f"\n{police_forces_result}")
    
except ApiError as e:
    logger.exception(f'API Error: \n{e}')


INFO:police forces with specified social engagement method:
                    id                                name
0    avon-and-somerset      Avon and Somerset Constabulary
1         bedfordshire                 Bedfordshire Police
2             cheshire               Cheshire Constabulary
3            cleveland                    Cleveland Police
4              cumbria                Cumbria Constabulary
5           derbyshire             Derbyshire Constabulary
6   devon-and-cornwall             Devon & Cornwall Police
7               dorset                       Dorset Police
8               durham                 Durham Constabulary
9          dyfed-powys                  Dyfed-Powys Police
10               essex                        Essex Police
11     gloucestershire        Gloucestershire Constabulary
12  greater-manchester           Greater Manchester Police
13               gwent                        Gwent Police
14           hampshire              Hampshire Constabul